# 도전학기 '빅데이터와 인공지능을 활용한 시스템 강건설계' 머신러닝 챌런지!

#### - 머신러닝 챌런지 목표 
: 실습내용을 바탕으로 직접 인공지능 모델을 학습시키는 코드를 작성하는 능력 평가


#### - 가이드에 따라 총 4단계의 머신러닝 챌런지를 수행하며, 각 단계의 결과값(csv 형태의 데이터 파일)을 1개 압축파일(.zip)로 제출
- 과제 수행을 위해 최소한 가이드가 있으나 가이드에 따르지 않고 다른 방법으로 코드 작성해도 결과만 맞으면 무관

#### - 데이터 출처
- 베어링 마모 상태에 따라 수집된 센서데이터 (데이터챌린지와 동일)
- 수집된 센서데이터의 특징 데이터 추출 및 t-test 완료
- P-value 상위 특징값 선택 단계부터 수행
- K-Fold 교차검증만 수행, 최종 모델 학습을 위한 전체 데이터 학습은 수행 X

.

.

.



### 필요한 라이브러리 import
- numpy, pandas 등 필요한 라이브러리들 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 데이터 불러오기
- FeatureNormal, FeatureAbnormal 및 P_value 불러오기

In [2]:
FeatureNormal   = pd.read_csv('ProcessedData/FeatureNormal', header = None)
FeatureAbnormal = pd.read_csv('ProcessedData/FeatureAbnormal', header = None)
P_value = pd.read_csv('ProcessedData/P_value', header = None)

# [1단계] P-value 기반 구분성(유의수준↓) 상위 특징값 5개 선택 (10점)

## 선택된 특징값은 하나의 데이터로 합치기

#### [필수] 레이블 이름(변수명) : 
- FeatureSelected

In [5]:
FeatureNormal.shape

(20, 150)

In [3]:
# 특징값 선택 코드 작성
Rank = 5
NoOfData = int(FeatureNormal.shape[1])
Normal   = np.zeros((Rank,NoOfData))
Abnormal = np.zeros((Rank,NoOfData))

for i in range(Rank):
    
    index         = int(P_value.iloc[i,0])
    Normal[i,:]   = FeatureNormal.iloc[index,:].values
    Abnormal[i,:] = FeatureAbnormal.iloc[index,:].values

# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(np.concatenate([Normal, Abnormal] , axis=1))

print("Selected Feature Data Size :", FeatureSelected.shape)
print("= 데이터 %d개(정상/고장 각 %d개씩)가 각각 %d개의 최종 선택된 특징값으로 구성됨"
      %(FeatureSelected.shape[1], FeatureSelected.shape[1]/2, FeatureSelected.shape[0]))

Selected Feature Data Size : (5, 300)
= 데이터 300개(정상/고장 각 150개씩)가 각각 5개의 최종 선택된 특징값으로 구성됨


In [4]:
FeatureSelected.shape

(5, 300)

### 1단계 결과물 제출용 데이터 파일로 저장 (수강생 번호 외 코드수정X)

- 위에서 데이터 이름(변수명) 가이드에 맞게 지정됐는지 재확인 요망
- FeatureSelected의 shape는 (5, 300)

In [16]:
StudentNo = 414   # 수강생 번호 입력

# 아래는 수정 금지
Path = './Result/ST%d_MC1'%StudentNo

FeatureSelected.to_csv(Path, sep=',' , header=None , index=None)

# [2단계] K-Fold 교차검증을 위한 Fold별 데이터 분할 생성 (5점)

## 3개의 Fold로 나누어 교차검증


#### [필수] 데이터 이름(변수명) : 
- Training_Fold1~3
- Validation_Fold1~3

In [6]:
NoOfData   = int(FeatureSelected.shape[1]/2)   # 데이터 개수 (정상/고장 각각)
Fold       = 3                   # Fold 개수 선정 : 데이터 개수의 약수여야 함.

FeatNo     = int(FeatureSelected.shape[0])  # 데이터 특징 수 (=데이터 차원)
FoldDataNo = int(NoOfData/Fold)            # 1개 Fold 당 (검증)데이터 개수

print(FeatNo, FoldDataNo)

5 50


In [7]:
NoOfData

150

In [8]:
NormalSet   = np.array(FeatureSelected.iloc[: , :NoOfData])
AbnormalSet = np.array(FeatureSelected.iloc[: , NoOfData:])
FeatureSelected_Reshaped = pd.DataFrame(np.concatenate([NormalSet , AbnormalSet] , axis=0))
FeatureSelected_Reshaped.shape

(10, 150)

In [9]:
q=np.zeros([1,3])
q

array([[0., 0., 0.]])

In [10]:
q.shape

(1, 3)

In [11]:
# Validation Data set
for i in range(Fold):
    
    temp_Valid_Normal   = FeatureSelected_Reshaped.iloc[:FeatNo , FoldDataNo*i : FoldDataNo*(i+1)]
    print(FoldDataNo*i)
    temp_Valid_Abnormal = FeatureSelected_Reshaped.iloc[FeatNo: , FoldDataNo*i : FoldDataNo*(i+1)]
    print(temp_Valid_Normal.shape)
    temp_Valid = pd.DataFrame(np.transpose(np.concatenate([temp_Valid_Normal, temp_Valid_Abnormal] , axis=1)))
    print(temp_Valid.shape)
    s = 'Validation_Fold%d = temp_Valid'%(i+1)
    exec(s)

    
# Training Data set
for i in range(Fold):
    
    temp_Train_Front = FeatureSelected_Reshaped.iloc[:,:FoldDataNo*i]
    temp_Train_Back  = FeatureSelected_Reshaped.iloc[:,FoldDataNo*(i+1):]
    temp_Train_Total = np.concatenate([temp_Train_Front , temp_Train_Back] , axis=1)
    temp_Train_Final = pd.DataFrame(np.transpose(np.concatenate([temp_Train_Total[:FeatNo,:],temp_Train_Total[FeatNo:,:]] , axis=1)))
    
    s ='Training_Fold%d  = temp_Train_Final'%(i+1)
    exec(s)

0
(5, 50)
(100, 5)
50
(5, 50)
(100, 5)
100
(5, 50)
(100, 5)


In [12]:
# 데이터 분할 코드 작성




In [13]:
print(Training_Fold1.shape, Validation_Fold1.shape)

(200, 5) (100, 5)


In [14]:
Training_Fold1.head()

,0,1,2,3,4
0,1.648684,-1.585628,0.130675,0.176980,0.014246
1,1.881879,-2.068165,0.128735,0.175475,0.014219
2,1.417367,-1.214995,0.127826,0.172634,0.013463
3,1.412485,-1.015596,0.131654,0.173713,0.012843
4,2.149997,-1.623931,0.130003,0.172117,0.012723


### 2단계 결과물 제출용 데이터 파일로 저장 (수강생 번호 외 코드수정X)

- 위에서 데이터 이름(변수명) 가이드에 맞게 지정됐는지 재확인 요망

In [26]:
# StudentNo = 0   # 수강생 번호 입력

# 아래는 수정 금지
Path = './Result/ST%d_MC2'%StudentNo

Training_Fold2.to_csv(Path, sep=',' , header=None , index=None)

# [3단계] 모델 성능 검증을 위한 레이블 생성 (5점)

## KNN, SVM 학습을 위한 레이블과 ANN 학습을 위한 레이블 모두 생성

- 정상 조건의 레이블 : 0 또는 [1,0]
- 고장 조건의 레이블 : 1 또는 [0,1]

#### [필수] 레이블 이름(변수명) : 
- TrainingFold_Label / ValidationFold_Label
- TrainingFold_Label_forANN / ValidationFold_Label_forANN

#### (1) KNN & SVM 레이블 생성

In [15]:
NoOfLabel_Train = int(Training_Fold1.shape[0]/2)
NoOfLabel_Valid = int(Validation_Fold1.shape[0]/2)

print(NoOfLabel_Train, NoOfLabel_Valid)


100 50


In [16]:
# 레이블 생성 코드 작성
TrainingFold_Label   = np.zeros(2*NoOfLabel_Train , dtype=int)
ValidationFold_Label = np.zeros(2*NoOfLabel_Valid , dtype=int)

# 고장데이터(학습용) Label 값 = 1
TrainingFold_Label[NoOfLabel_Train:] = 1

# 고장데이터(검증용) Label 값 = 1

ValidationFold_Label[NoOfLabel_Valid:] = 1

TrainingFold_Label   = pd.Series(TrainingFold_Label)
ValidationFold_Label = pd.Series(ValidationFold_Label)

#TrainingFold_Label
ValidationFold_Label



0     0
1     0
2     0
3     0
4     0
     ..
95    1
96    1
97    1
98    1
99    1
Length: 100, dtype: int32

#### (2) ANN 레이블 생성

In [17]:
# 레이블 생성 코드 작성
TrainingFold_Label_forANN   = np.zeros((NoOfLabel_Train*2,2) , dtype=int)
ValidationFold_Label_forANN = np.zeros((NoOfLabel_Valid*2,2) , dtype=int)

TrainingFold_Label_forANN[:NoOfLabel_Train , 0] = 1    # 정상(학습)데이터 레이블 : [1,0]
TrainingFold_Label_forANN[NoOfLabel_Train: , 1] = 1    # 고장(학습)데이터 레이블 : [0,1]

ValidationFold_Label_forANN[:NoOfLabel_Valid , 0] = 1  # 정상(검증)데이터 레이블 : [1,0]
ValidationFold_Label_forANN[NoOfLabel_Valid: , 1] = 1  # 고장(검증)데이터 레이블 : [0,1]

TrainingFold_Label_forANN   = pd.DataFrame(TrainingFold_Label_forANN)
ValidationFold_Label_forANN = pd.DataFrame(ValidationFold_Label_forANN)

TrainingFold_Label_forANN
#ValidationFold_Label_forANN



,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
195,0,1
196,0,1
197,0,1
198,0,1


### 3단계 결과물 제출용 데이터 파일로 저장 (수강생 번호 외 코드수정X)

- 위에서 데이터 이름(변수명) 가이드에 맞게 지정됐는지 재확인 요망

In [31]:
# StudentNo = 0   # 수강생 번호 입력

# 아래는 수정 금지
Path1 = './Result/ST%d_MC3_1'%StudentNo
Path2 = './Result/ST%d_MC3_2'%StudentNo


ValidationFold_Label.to_csv(     Path1, sep=',' , header=None , index=None)
TrainingFold_Label_forANN.to_csv(Path2, sep=',' , header=None , index=None)

# [4단계] 모델 학습 후 모델 검증 정확도 결과 확인 (10점)

## KNN, SVM, ANN 각각 모델 검증 후 정확도를 DataFrame 형태로 저장 후 제출

- 해당 DataFrame의 shape는 (3, 3)
- 1열 : KNN 정확도 (행은 각 Fold 순서)
- 2열 : SVM 정확도 (행은 각 Fold 순서)
- 3열 : ANN 정확도 (행은 각 Fold 순서)

#### [필수] DataFrame 이름(변수명) : 
- Total_Accuracy

In [18]:
from sklearn.neighbors import KNeighborsClassifier
import joblib

In [103]:
Total_Accuracy = np.zeros((3, 3))

#### (1) KNN 학습 및 검증
- KNN Parameter : n_neighbors = 7

In [104]:
a=np.zeros((2,2))

In [105]:
a

array([[0., 0.],
       [0., 0.]])

In [106]:
a[0,1]=0

In [107]:
a

array([[0., 0.],
       [0., 0.]])

In [108]:
ValidationFold_Label

0     0
1     0
2     0
3     0
4     0
     ..
95    1
96    1
97    1
98    1
99    1
Length: 100, dtype: int32

In [109]:
Validation_Fold1

,0,1,2,3,4
0,1.546920,-1.403503,0.138501,0.192596,0.017911
1,1.908916,-1.555211,0.131867,0.186068,0.017233
2,2.373429,-1.835721,0.128184,0.181971,0.016682
3,2.004297,-1.683262,0.123096,0.177577,0.016381
4,1.680228,-1.104218,0.120492,0.172376,0.015195
...,...,...,...,...,...
95,7.020054,-6.050879,0.149866,0.727640,0.507000
96,7.560045,-6.292711,0.147853,0.729995,0.511033
97,7.914156,-6.430150,0.152176,0.734719,0.516654
98,6.315587,-6.404615,0.152742,0.737743,0.520935


In [110]:
TrainingFold_Label

0      0
1      0
2      0
3      0
4      0
      ..
195    1
196    1
197    1
198    1
199    1
Length: 200, dtype: int32

In [111]:
Training_Fold1.shape

(200, 5)

In [112]:
TrainingFold_Label.shape

(200,)

In [113]:
# KNN 코드 작성
for i in range(Fold):
    c1 = 'Training_CurrentFold = Training_Fold%d'%(i+1)
    exec(c1)
    c2 = 'Validation_CurrentFold = Validation_Fold%d'%(i+1)
    exec(c2)    

    knnModel_CurrentFold = KNeighborsClassifier(n_neighbors = 7).fit(Training_CurrentFold,TrainingFold_Label)
    knnscore_Fold= knnModel_CurrentFold.score(Validation_CurrentFold , ValidationFold_Label)

    print(knnscore_Fold)
    
    Total_Accuracy[i,0]=knnscore_Fold*100






1.0
1.0
1.0


In [114]:
Total_Accuracy

array([[100.,   0.,   0.],
       [100.,   0.,   0.],
       [100.,   0.,   0.]])

#### (2) SVM 학습 및 검증
- SVM Parameter : kernel = sigmoid

In [115]:
from sklearn import svm,metrics

In [116]:
# SVM 코드 작성
Add    = 0
Divide = 0

for i in range(Fold):
    c1 = 'Training_CurrentFold = Training_Fold%d'%(i+1)
    exec(c1)
    c2 = 'Validation_CurrentFold = Validation_Fold%d'%(i+1)
    exec(c2)    
    
    svmModel_CurrentFold = svm.SVC(kernel='linear')
    svmModel_CurrentFold.fit(Training_CurrentFold , TrainingFold_Label)
    Predicted = np.array(svmModel_CurrentFold.predict(Validation_CurrentFold))
    
    svmscore_Fold = metrics.accuracy_score(ValidationFold_Label , Predicted)
    exec(c3)

    Total_Accuracy[i,1]=svmscore_Fold*100







In [117]:
Total_Accuracy

array([[100., 100.,   0.],
       [100., 100.,   0.],
       [100., 100.,   0.]])

#### (3) ANN 학습 및 검증

##### [필수] 재현성을 위해 for구문 내 seed를 777로 고정

ANN Parameter
- learningRate : 0.01
- noOfNeuron   : 8
- iteration    : 3
- Hidden Layer 수 : 1개

In [118]:
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(777)

In [119]:
# ANN 코드 작성
learningRate  = 0.01
noOfNeuron    = 8
iteration     = 3
def ANN_model(input_data):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units = noOfNeuron,
                                 input_shape = (input_data.shape[1],), activation = 'relu'))    # Hidden Layer 1
    model.add(keras.layers.Dense(units = 2,                            activation = 'softmax')) # Output Layer
    
    
    
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model




In [120]:
Label        = TrainingFold_Label_forANN
Label_Val    = ValidationFold_Label_forANN
Accuracy_sum = 0

print('[Reslut of K-fold Cross Validation] \n')

for i in range(Fold):
    tf.random.set_seed(777)
    
    s1= 'Data     = Training_Fold%d'  %(i+1)
    s2= 'Data_Val = Validation_Fold%d'%(i+1)
    exec(s1)
    exec(s2)
    
    model = ANN_model(Data)
    
# train my model
    hist = model.fit(Data, Label, epochs=iteration, verbose = 0)
    Loss, Accuracy = model.evaluate(Data_Val,  Label_Val, verbose=0)
    Accuracy_sum = Accuracy_sum + Accuracy
    print('Fold {}: {:.2f}%'.format((i+1), Accuracy*100))
    Total_Accuracy[i,2]=Accuracy*100

print('* Average accuracy : {:.2f}%'.format((Accuracy_sum/Fold)*100))

[Reslut of K-fold Cross Validation] 

Fold 1: 50.00%
Fold 2: 50.00%
Fold 3: 50.00%
* Average accuracy : 50.00%


In [121]:
Total_Accuracy = pd.DataFrame(Total_Accuracy)
Total_Accuracy

,0,1,2
0,100.0,100.0,50.0
1,100.0,100.0,50.0
2,100.0,100.0,50.0


### 4단계 결과물 제출용 데이터 파일로 저장 (수강생 번호 외 코드수정X)

- 위에서 데이터 이름(변수명) 가이드에 맞게 지정됐는지 재확인 요망

In [122]:
StudentNo = 414   # 수강생 번호 입력

# 아래는 수정 금지
Path = './Result/ST%d_MC4'%StudentNo

Total_Accuracy.to_csv(Path, sep=',' , header=None , index=None)

.

.

.



# * 결과 데이터(MC1~4)를 하나의 zip파일로 압축하여 제출

### >>> 압축파일 이름 ST(수강생 번호)_MC (예시 : 'ST000_MC.zip'  //  'ST00_MC.zip'  //  'ST0_MC.zip')

.

.

.



수고하셨습니다!